In [1]:
import pickle
with open('label.pkl', 'rb') as f:
      dt = pickle.load(f)

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000000)

In [3]:
dt

,Tweet
0,hell yeah trump got impeached looks like he is finally out of office deletes tweet minutes later wow none of you know what impeachment means the senate still has to vote before he is removed from office go take a college course
1,breaking news trump is committing felony bribery by giving fundraising cash to gop senators ahead of impeachment trial this can not be ignored
2,help me here I am being sincere this is still your guy forget the news source forget that this part of a pattern forget the lies the bluster forget the impeachment stuff I am curious why you are still behind someone who consistently behaves like this help me
3,i hate to break the news to the resistance but you have zero leverage you can keep your partisan impeachment articles as long as you want senate will just keep confirming judges approving trade deals doing the peoples business
4,impeachment should not be a happy thing it should be a somber time this is sick and proves what the republicicans have been saying all along impeachment notmyimpeachment trump landslide
...,...
486135,you all help me out by following my beautiful wife of over years deb is back to support she does fo
486136,christianity today s editorial board decided to play judge jury on potus based on a partisan impeachment i do not al
486137,the fulani jihadist in nigeria financed hillary clinton against trump they failed then they sent a sudanese impostor robot
486138,in my view moscowmitch is a f ing ruble loving coward the sniveling baby would do anything to keep his little j


In [4]:
# Module 2 (Label this data)
# Use snorkel
from snorkel.labeling import labeling_function
from snorkel.preprocess import preprocessor
import re
from textblob import TextBlob  

In [5]:
# Create labelling function based on words in the corpus
from snorkel.labeling import LabelingFunction
PWORDS = r"""\b(notmyimpeachment|do not impeach|meaningless impeachment|unfair|fair trial|not enough evidence|miss conduct|delay trial|fake impeachment)"""
def pos(sample):
        return 1 if re.search(PWORDS, str(sample)) else -1

NWORDS = r"\b(hell yeah|bribery|not happy|less moral|impeach trump|impeach our president)"
def neg(sample):
    return 0 if re.search(NWORDS, str(sample)) else -1

In [6]:
positive = LabelingFunction(f"positive", f=pos)

In [7]:
negative = LabelingFunction(f"negative", f=neg)

In [8]:
df_train = dt[0:450000]

In [9]:
# Create the labeling functions using the textblob sentiment analyzer
@preprocessor(memoize=True)
def textblob_polarity(x):
    scores = TextBlob(x.Tweet)
    x.polarity = scores.polarity
    return x


# Label high polarity tweets as positive.
@labeling_function(pre=[textblob_polarity])
def polarity_positive(x):
    return 1 if x.polarity > 0.2 else -1


# Label low polarity tweets as negative.
@labeling_function(pre=[textblob_polarity])
def polarity_negative(x):
    return 0 if x.polarity < -0.25 else -1


# Similar to polarity_negative, but with higher coverage and lower precision.
@labeling_function(pre=[textblob_polarity])
def polarity_negative_2(x):
    return 0 if x.polarity <= 0.2 else -1


In [10]:
from snorkel.labeling import PandasLFApplier
wlfs = [positive,negative]
blob_lfs = [polarity_positive, polarity_negative,polarity_negative_2]
lfs = wlfs + blob_lfs 
applier = PandasLFApplier(lfs)
L_train = applier.apply(df_train)

/home/seiryu/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 450000/450000 [07:15<00:00, 1033.28it/s]


In [11]:
# Analyse the coverage and accuracy
from snorkel.labeling import LFAnalysis
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
positive,0,[1],0.020802,0.020802,0.014662
negative,1,[0],0.017293,0.017293,0.001549
polarity_positive,2,[1],0.168544,0.007593,0.001453
polarity_negative,3,[0],0.092987,0.092987,0.004331
polarity_negative_2,4,[0],0.831456,0.117869,0.014636


In [12]:
print(f"Training set coverage: {100 * LFAnalysis(L_train).label_coverage(): 0.001f}%")

Training set coverage:  100.0%


In [13]:
from snorkel.labeling import LabelModel

# Train LabelModel.
label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=150, seed=125, log_freq=30, l2=0.1, lr=0.01)

In [14]:
label = label_model.predict(L_train)

In [15]:
label[1:100]

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [16]:
with open('flabel.pkl', 'wb') as f:
     pickle.dump(label, f)